In [1]:
from datasets import load_dataset
import torch
import transformers
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset(
    "csv",
    data_files={
        "train": "./hf_train.csv",
        "valid": "./hf_valid.csv",
        #"test": "./test.csv",
    },
)

checkpoint = "bionlp/bluebert_pubmed_mimic_uncased_L-24_H-1024_A-16"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Using custom data configuration default-fd5364d0717a7729
Reusing dataset csv (C:\Users\170890\.cache\huggingface\datasets\csv\default-fd5364d0717a7729\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def tokenize_function(exam):
    return tokenizer(exam["sentence"], truncation=True, padding='max_length')

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

  0%|          | 0/37 [00:00<?, ?ba/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/10 [00:00<?, ?ba/s]

In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
train_args = TrainingArguments(
    "test-trainer",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 5,
    learning_rate = 2e-5,
    weight_decay = 0.01,
)

Some weights of the model checkpoint at bionlp/bluebert_pubmed_mimic_uncased_L-24_H-1024_A-16 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequ

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    train_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["valid"],
    data_collator = data_collator,
    tokenizer = tokenizer,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Unnamed: 0, sentence.
***** Running training *****
  Num examples = 36754
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11490
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: sangyunlee (use `wandb login --relogin` to force relogin)


Step,Training Loss
500,0.155100


Saving model checkpoint to test-trainer\checkpoint-500
Configuration saved in test-trainer\checkpoint-500\config.json
Model weights saved in test-trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-500\special_tokens_map.json
